In [ ]:
#pip install together
#pip install langchain_together
from langchain_together import TogetherEmbeddings
import pandas as pd
import numpy as np

df = pd.read_excel('reviews_data.xlsx')
df
import os
import getpass
if not os.getenv("TOGETHER_API_KEY"):
    os.environ["TOGETHER_API_KEY"] = "cb6bcac5583a1f6ce6bbbf1eb1af471d5bbeaf1c4ee6551e1d830a2b0cd00964"

# Initialize the TogetherEmbeddings model
embeddings = TogetherEmbeddings(
    model="togethercomputer/m2-bert-80M-8k-retrieval"  # Example model; check docs for Llama-based alternatives
)
pip install pinecone
# Create embeddings for all reviews - this will take close to 1.5 - 2 hours
reviews = df["Review"].tolist()

# Process embeddings in batches
embedding_list = []
for i in range(0, len(reviews), 128):
    batch = reviews[i : i + 128]  # Get batch
    batch_embeddings = embeddings.embed_documents(batch)  # Generate embeddings for batch
    embedding_list.extend(batch_embeddings)  # Store results
    print(f"Processed {i + len(batch)} / {len(reviews)} reviews")
metadata_list = df.apply(lambda row: {
    "customer_id": int(row["customer_id"]),
    "review_date": row["review_date_numeric"],
    "Rating": int(row["Rating"]),
    "review_id":row['review_id']
}, axis=1).tolist()
metadata_list[0]
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(
    api_key='pcsk_4SKEDS_QZVnzA5gEJ6c58zYvFRbQLJTTXasgwVHC9hX428L4ToFtxWpyTGJ9UmEqhzvWEc'
)
# Check whether any indexes already exist
pc.list_indexes()
# create the index
pc.create_index(
    name='hotel-reviews',
    dimension=768,
    metric='cosine',
    deletion_protection='enabled',
    spec=ServerlessSpec(
        cloud='aws',
        region='us-east-1'
    )
)
index = pc.Index(host="hotel-reviews-dcl6jm7.svc.aped-4627-b74a.pinecone.io")
vectors = [
    (str(i), embedding, metadata_list[i])
    for i, embedding in enumerate(embedding_list)
]
# Insert embeddings + metadata into the index
batch_size = 100  # Adjust this number as needed.

for i in range(0, len(embedding_list), batch_size):
    batch_vectors = [
        (str(i + j), embedding_list[i + j], metadata_list[i + j])
        for j in range(min(batch_size, len(embedding_list) - i))
    ]
    index.upsert(vectors=batch_vectors)
    print(f"Upserted batch from {i} to {i + len(batch_vectors)}")
# experiment with below parts
query_embedding = embeddings.embed_query("What are some of the reviews that mention restaurant, food, lunch, breakfast, dinner")
results = index.query(
    vector=query_embedding,
    top_k=5,
    namespace="",
    include_metadata=True,
    filter={
        "Rating": {"$lte": 9},
        "review_date": {"$gte": 20240101, "$lte": 20240108}
    }
)
matches = results["matches"]

# Extract review_ids from the matches (convert them to int if necessary)
matched_ids = [int(match["metadata"]["review_id"]) for match in matches]
matched_ids
req_df = df[df["review_id"].isin(matched_ids)]
req_df['Review']
concatenated_reviews = " ".join(req_df["Review"].tolist())
concatenated_reviews
from together import Together
import os
client = Together()
response = client.chat.completions.create(
    model="meta-llama/Llama-Vision-Free",
    messages = [{"role": "user", "content": f"""Briefly Summarize the overall sentiment of customers about food and restaurant based on these reviews - {
    concatenated_reviews}. Dont mention the name of the hotel"""}]
)
print(response.choices[0].message.content)


